In [2]:
pip install requests beautifulsoup4

In [9]:
import requests
from bs4 import BeautifulSoup

# 멜론 차트 URL
url = 'https://www.melon.com/chart/index.htm'

# 멜론 사이트에 접근할 때 필요한 헤더 정보
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# 멜론 차트 페이지 요청
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# 곡 정보가 있는 테이블 선택
songs = soup.select('tr[data-song-no]')

# 장르에 따른 감정 매핑
genre_emotion_mapping = {
    '댄스': '기쁨',
    '발라드': '기쁨',
    '국내드라마': '기쁨',
    '블루스': '슬픔',
    'R&B/Soul': '슬픔',
    '랩/힙합': '분노',
    '록/메탈': '분노',
    '성인가요/트로트': '불안',
    '인디음악': '불안'
}

def get_emotion_by_genre(genre):
    for key in genre_emotion_mapping:
        if key in genre:
            return genre_emotion_mapping[key]
    return '불안'

# 각 곡의 제목과 장르 추출
for song in songs:
    title = song.select_one('div.ellipsis.rank01 > span > a').text
    song_id = song['data-song-no']

    # 각 곡의 상세 페이지 URL
    song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
    song_response = requests.get(song_url, headers=headers)
    song_soup = BeautifulSoup(song_response.content, 'html.parser')

    # 장르 추출
    genre = song_soup.select_one('div.meta > dl > dd:nth-of-type(3)').text
    emotion = get_emotion_by_genre(genre)

    print(f'{title} {genre} {emotion}')

Supernova 댄스 기쁨
How Sweet 댄스 기쁨
Bubble Gum 댄스 기쁨
SPOT! (feat. JENNIE) 랩/힙합 분노
해야 (HEYA) 댄스 기쁨
소나기 발라드, 국내드라마 기쁨
고민중독 록/메탈 분노
Magnetic 댄스 기쁨
Armageddon 댄스 기쁨
나는 아픈 건 딱 질색이니까 댄스 기쁨
첫 만남은 계획대로 되지 않아 댄스 기쁨
미안해 미워해 사랑해 발라드, 국내드라마 기쁨
천상연 발라드 기쁨
SHEESH R&B/Soul 슬픔
한 페이지가 될 수 있게 록/메탈 분노
예뻤어 록/메탈 분노
Love wins all 발라드 기쁨
밤양갱 R&B/Soul 슬픔
에피소드 록/메탈 분노
사랑은 늘 도망가 발라드, 국내드라마 기쁨
비의 랩소디 발라드 기쁨
MAESTRO 댄스 기쁨
Hype Boy 댄스 기쁨
우리들의 블루스 발라드 기쁨
그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection)) 발라드 기쁨
Drama 댄스 기쁨
To. X R&B/Soul 슬픔
온기 발라드 기쁨
Welcome to the Show 록/메탈 분노
모래 알갱이 발라드 기쁨
I AM 댄스 기쁨
ETA 댄스 기쁨
Home 댄스 기쁨
Seven (feat. Latto) - Clean Ver. 댄스 기쁨
다시 만날 수 있을까 발라드 기쁨
헤어지자 말해요 발라드 기쁨
Super Shy 댄스 기쁨
EASY 랩/힙합 분노
이제 나만 믿어요 성인가요/트로트 불안
Do or Die 댄스 기쁨
그랬나봐 발라드, 국내드라마 기쁨
Smart R&B/Soul 슬픔
Ditto 댄스 기쁨
무지개 록/메탈 분노
사랑인가 봐 발라드, 국내드라마 기쁨
London Boy 록/메탈 분노
Spicy 댄스 기쁨
Polaroid 록/메탈 분노
Midas Touch 댄스 기쁨
Girls Never Die 댄스 기쁨
봄눈 발라드, 국내드라마 기쁨
인사 발라드, 인디음악 기쁨
슬픈 초대장 발라드, 인디음악 기쁨
Love 119 댄스 기쁨
Lucky Girl Syndrome 댄스 기쁨
MANIAC

In [10]:
pip install pandas

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from googleapiclient.discovery import build

# 멜론 차트 URL
url = 'https://www.melon.com/chart/index.htm'

# 멜론 사이트에 접근할 때 필요한 헤더 정보
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# 멜론 차트 페이지 요청
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# 곡 정보가 있는 테이블 선택
songs = soup.select('tr[data-song-no]')

# 장르에 따른 감정 매핑
genre_emotion_mapping = {
    '댄스': '기쁨',
    '발라드': '기쁨',
    '국내드라마': '기쁨',
    '블루스': '슬픔',
    'R&B/Soul': '슬픔',
    '랩/힙합': '분노',
    '록/메탈': '분노'
}

def get_emotion_by_genre(genre):
    for key in genre_emotion_mapping:
        if key in genre:
            return genre_emotion_mapping[key]
    return '불안'

# YouTube API 설정
DEVELOPER_KEY = 'YOUR_YOUTUBE_API_KEY'  # 여기에 본인의 YouTube API 키를 입력하세요.
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

def get_youtube_link(query):
    search_response = youtube.search().list(
        q=query,
        part='snippet',
        maxResults=1
    ).execute()

    if search_response['items']:
        video_id = search_response['items'][0]['id']['videoId']
        return f'https://www.youtube.com/watch?v={video_id}'
    else:
        return 'No link found'

# 곡 정보 저장을 위한 리스트
song_data = []

# 각 곡의 제목, 가수, 장르 추출 및 유튜브 링크 추가
for song in songs:
    title = song.select_one('div.ellipsis.rank01 > span > a').text
    artist = song.select_one('div.ellipsis.rank02 > a').text
    song_id = song['data-song-no']

    # 각 곡의 상세 페이지 URL
    song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
    song_response = requests.get(song_url, headers=headers)
    song_soup = BeautifulSoup(song_response.content, 'html.parser')

    # 장르 추출
    genre = song_soup.select_one('div.meta > dl > dd:nth-of-type(3)').text
    emotion = get_emotion_by_genre(genre)

    # 유튜브 링크 추출
    youtube_link = get_youtube_link(f'{title} {artist}')

    # 곡 정보 리스트에 추가
    song_data.append({
        'Title': title,
        'Artist': artist,
        'Genre': genre,
        'Emotion': emotion,
        'YouTube Link': youtube_link
    })

# 데이터프레임으로 변환
df = pd.DataFrame(song_data)

# 데이터프레임 출력
print(df)

HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/search?q=Supernova+aespa&part=snippet&maxResults=1&key=YOUR_YOUTUBE_API_KEY&alt=json returned "API key not valid. Please pass a valid API key.". Details: "[{'message': 'API key not valid. Please pass a valid API key.', 'domain': 'global', 'reason': 'badRequest'}]">